In [40]:
import os
import json
from pprint import pprint
import pandas as pd
pd.set_option('display.max_colwidth', -1)

from dataservice.util.data_import.utils import (
    read_json, 
    write_json, 
    cols_to_lower, 
    dropna_rows_cols,
    reformat_column_names
)

DATA_DIR = '/Users/singhn4/Projects/kids_first/data/Chung'
DBGAP_DIR = os.path.join(DATA_DIR, 'dbgap')
MANIFESTS_DIR = os.path.join(DATA_DIR, 'manifests')

In [41]:
# Create study
study = {
    'data_access_authority': 'dbGaP',
    'study_id': 'phs001110',
    'study_version': 'v1.p1',
    'study_name': 'Genomic Analysis of Congenital Diaphragmatic Hernia and Associated Congenital Anomalies',
    'attribution': 'https://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/study.cgi?study_id=phs001110.v1.p1'
}
study_df = pd.DataFrame([study])
study_df.to_csv(os.path.join(DATA_DIR, 'study.txt'))

# Create investigator
invest = {
    'investigator_name': 'Wendy Chung',
    'institution': 'Columbia University Health Sciences'
}
inv_df = pd.DataFrame([invest])
inv_df.to_csv(os.path.join(DATA_DIR, 'investigator.txt'))

In [3]:
@reformat_column_names
@dropna_rows_cols
def read_study_file_data(filepaths=None):
    """
    Read in raw study files
    """
    if not filepaths:
        filepaths = os.listdir(DBGAP_DIR)
        filepaths.extend(os.listdir(MANIFESTS_DIR))

    study_files = [{"study_file_name": f}
                   for f in filepaths]
    return pd.DataFrame(study_files)

@reformat_column_names
@dropna_rows_cols
def read_study_data(filepath=None):
    """
    Read study data
    """
    if not filepath:
        filepath = os.path.join(DATA_DIR,
                                'study.txt')
    df = pd.read_csv(filepath)

    return df

@reformat_column_names
@dropna_rows_cols
def read_investigator_data(filepath=None):
    """
    Read investigator data
    """
    if not filepath:
        filepath = os.path.join(DATA_DIR,
                                'investigator.txt')
    df = pd.read_csv(filepath)

    return df

@reformat_column_names
@dropna_rows_cols
def read_subject_data(filepath=None):
    """
    Read subject data file
    """
    if not filepath:
        filepath = os.path.join(DBGAP_DIR, '4a_dbGaP_SubjectDS_corrected_7-16.xlsx')
    df = pd.read_excel(filepath, dtype={'SUBJECT_ID': str})
    # Decode consent ints to consent strings
    def func(row): 
        _map = {1: "Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)"}
        return _map.get(row['CONSENT'])
    df['CONSENT'] = df.apply(func, axis=1)

    # Decode affected status ints to strings
    def func(row): 
        _map = {0:'unknown', 1: "affected", 2: "unaffected"}
        return _map.get(row['AFFECTED_STATUS'])
    df['AFFECTED_STATUS'] = df.apply(func, axis=1)
    return df

@reformat_column_names
@dropna_rows_cols
def read_subject_attr_data(filepath=None):
    """
    Read subject attributes file
    """
    if not filepath:
        filepath = os.path.join(DBGAP_DIR, '3a_dbGaP_SubjectAttributesDS_corrected.6.12.xlsx')
    df = pd.read_excel(filepath, dtype={'SUBJECT_ID': str})
    # Decode body_site chars to strings
    def func(row): 
        _map = {'B':'blood', 'SK': 'skin', 'D': 'diaphragm', 'SV': 'saliva', 'A': 'amniocytes', 'M': 'amniocytes'}
        return _map.get(row['body_site'])
    df['body_site'] = df.apply(func, axis=1)
    return df

@reformat_column_names
@dropna_rows_cols
def read_family_data(filepath=None):
    """
    Read pedigree data
    """
    if not filepath:
        filepath = os.path.join(DBGAP_DIR, '6a_dbGaP_PedigreeDS_corrected.6.12.xlsx')
    df = pd.read_excel(filepath)
    del df['SEX']
    
    return df

@reformat_column_names
@dropna_rows_cols
def read_sample_manifests():
    """
    Read and combine all sample manifest sheets
    """
    # Sample manifests
    # Combine all sample manifest sheets
    dfs = [pd.read_excel(os.path.join(MANIFESTS_DIR, filename))

           for filename in os.listdir(MANIFESTS_DIR)

           ]
    df = pd.concat(dfs)
    df = df[df['Sample ID'].notnull()]
    
    df.rename(columns={'Alias.2': 'is_proband'}, inplace=True)
    
    return df[['Concentration', 'Volume', 'Sample ID', 'Sample Type', 'is_proband']]

@reformat_column_names
@dropna_rows_cols
def read_subject_sample_data(filepath=None):
    if not filepath:
        filepath = os.path.join(DBGAP_DIR, '5a_dbGaP_SubjectSampleMappingDS cumulative.xlsx')
    return pd.read_excel(filepath, delimiter='\t')

@reformat_column_names
@dropna_rows_cols
def read_demographic_data(filepath=None):
    """
    Read demographic data from phenotype file
    """
    if not filepath:
        filepath = os.path.join(DBGAP_DIR, '2a_dbGaP_SubjectPhenotypesDS.xlsx')
    df = pd.read_excel(filepath)
    # Make all values lower case
    for col in ['Ethnicity', 'Race']:
        df[col] = df[col].apply(lambda x: str(x).lower().strip())
    return df[['SUBJECT_ID', 'SEX', 'Ethnicity', 'Race']]

@reformat_column_names
@dropna_rows_cols
def read_phenotype_data(filepath=None):
    """
    Read phenotype file and insert HPO IDs
    """
    if not filepath:
        filepath = os.path.join(DBGAP_DIR, '2a_dbGaP_SubjectPhenotypesDS.xlsx')

    df = pd.read_excel(filepath)
    df.drop(['Ethnicity', 'Race', 'SEX', 'discharge_status', 'ISOLATED'], inplace=True, axis=1)
    # Reshape to build the phenotypes df
    cols = df.columns.tolist()[1:]
    phenotype_df = pd.melt(df, id_vars='SUBJECT_ID', value_vars=cols,
                           var_name='phenotype', value_name='value')

    # Drop rows where value is NaN
    phenotype_df = phenotype_df[pd.notnull(phenotype_df['value'])]

    # Decode phenotypes to descriptive strings
    def func(row):
        _map = {0: 'no', 1: 'yes'}
        return _map.get(row['value'], row['value'])
    phenotype_df['value'] = phenotype_df.apply(func, axis=1)

    # Decode phenotypes to descriptive strings
    def func(row):
        # Always take most specific value
        if row['value'] not in ['yes', 'no']:
            val = row['value']
        else:
            _map = {'CHD': 'congenital heart defect', 'CNS': 'central nervous system defect', 
                'GI': 'gastrointestinal defect'}
            val = _map.get(row['phenotype'], 'congenital birth defect')
        return val
    phenotype_df['phenotype'] = phenotype_df.apply(func, axis=1)

    # Set observed
    phenotype_df['observed'] = phenotype_df['value'].apply(lambda x: 'positive' if x != 'no' else 'negative')
    del phenotype_df['value']
    
    # Add HPOs
    from dataservice.util.data_import.etl.hpo import mapper
    hpo_mapper = mapper.HPOMapper(DATA_DIR)
    phenotype_df = hpo_mapper.add_hpo_id_col(phenotype_df)
    return phenotype_df

@reformat_column_names
@dropna_rows_cols
def read_outcome_data(filepath=None):
    """
    Read outcome data from phenotype file
    """
    if not filepath:
        filepath = os.path.join(DBGAP_DIR, '2a_dbGaP_SubjectPhenotypesDS.xlsx')
    df = pd.read_excel(filepath)
    
    # Replace NaN values with None
    df['discharge_status'] = df['discharge_status'].where((pd.notnull(df['discharge_status'])),999)
    
    # Map discharge status
    # 1=Alive 4=Deceased 0=Fetal sample 8=unknown NA=Not applicable
    def func(row): 
        _map = {0:'alive', 1: 'deceased', 4:'fetal sample', 8: 'unknown'}
        return _map.get(int(row['discharge_status']), 'not applicable')
    df['discharge_status'] = df.apply(func, axis=1)
    return df[['SUBJECT_ID', 'discharge_status']]

## Explore

In [4]:
# Db gap files
files = {f:os.path.join(DBGAP_DIR, f) for f in os.listdir(DBGAP_DIR)}
pprint(list(files.keys()))

['2a_dbGaP_SubjectPhenotypesDS.xlsx',
 '2b_dbGaP_SubjectPhenotypesDD.xlsx',
 '3a_dbGaP_SubjectAttributesDS_corrected.6.12.xlsx',
 '3b_dbGaP_SubjectAttributesDD.xlsx',
 '4a_dbGaP_SubjectDS_corrected_7-16.xlsx',
 '4b_dbGaP_SubjectDD_corrected_6_12.xlsx',
 '5a_dbGaP_SubjectSampleMappingDS cumulative.xlsx',
 '5b_dbGaP_SubjectSampleMappingDD.xlsx',
 '6a_dbGaP_PedigreeDS_corrected.6.12.xlsx',
 '6b_dbGaP_PedigreeDD.xlsx']


### Subject 

In [5]:
# Description of data
df1 = pd.read_excel(files['4b_dbGaP_SubjectDD_corrected_6_12.xlsx'], delimiter='\t', dtype={'SUBJID': str})
df1

,VARNAME,VARDESC,DOCFILE,TYPE,VARIABLE_SOURCE,SOURCE_VARIABLE_ID,COLLINTERVAL,VALUES,Unnamed: 8
0,SUBJECT_ID,De-identified Subject ID,NaN,encoded value,NaN,NaN,Collected at enrollment,NaN,NaN
1,CONSENT,Consent group as determined by DAC,NaN,encoded value,NaN,NaN,Collected at enrollment,"1=Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",NaN
2,AFFECTED_STATUS,Case control status of the subject for congenital diaphragmatic hernia/defects,NaN,encoded value,NaN,NaN,Collected at enrollment,1=affected,2=unaffected


In [6]:
# Subject data
df = pd.read_excel(files['4a_dbGaP_SubjectDS_corrected_7-16.xlsx'], delimiter='\t', dtype={'SUBJID': str})
df.head()

,SUBJECT_ID,CONSENT,AFFECTED_STATUS
0,06-0015,1,1
1,07-0016,1,1
2,09-0019,1,1
3,6-15F,1,2
4,6-15M,1,2


In [7]:
print(df['CONSENT'].unique())
print(df['AFFECTED_STATUS'].unique())
df.describe(include=['O']).T.sort_values('unique', ascending=False)

[1]
[1 2]


,count,unique,top,freq
SUBJECT_ID,962,962,CDH14-7F,1


In [8]:
# Subject attributes data description
df = pd.read_excel(files['3b_dbGaP_SubjectAttributesDD.xlsx'])
df

,VARNAME,VARDESC,DOCFILE,TYPE,VARIABLE_SOURCE,SOURCE_VARIABLE_ID,COLLINTERVAL,VALUES,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,SAMPLE_ID,De-identified Sample ID,NaN,string,NaN,NaN,Collected at enrollment,NaN,NaN,NaN,NaN,NaN,NaN
1,body_site,Body site where sample was collected,NaN,encoded value,NaN,NaN,Collected at enrollment,B=blood,SK=skin,D=diaphragm,SV=saliva,A=amniocytes,M=muscle
2,analyte_type,Analyte Type,NaN,encoded value,NaN,NaN,Collected at enrollment,NaN,NaN,NaN,NaN,NaN,NaN
3,is_tumor,Tumor status,NaN,encoded value,NaN,NaN,Collected at enrollment,Y=Is Tumor,N=Is not a tumor,NaN,NaN,NaN,NaN
4,SUBJECT_ID,De-identified Subject ID,NaN,string,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Subject attributes
df = pd.read_excel(files['3a_dbGaP_SubjectAttributesDS_corrected.6.12.xlsx'], delimiter='\t')
df.head()

,SUBJECT_ID,SAMPLE_ID,body_site,analyte_type,is_tumor
0,06-0015,SM-DH4NH,D,DNA,N
1,07-0016,SM-DH4NJ,B,DNA,N
2,09-0019,SM-DH4NK,SK,DNA,N
3,6-15F,SM-DH4NF,B,DNA,N
4,6-15M,SM-DH4NG,B,DNA,N


In [10]:
print(df.body_site.unique())
df.describe(include=['O']).T.sort_values('unique', ascending=False)

['D' 'B' 'SK' 'M' 'A']


,count,unique,top,freq
SUBJECT_ID,962,962,CDH14-7F,1
SAMPLE_ID,962,962,SM-DH5AX,1
body_site,962,5,B,945
analyte_type,962,1,DNA,962
is_tumor,962,1,N,962


### Family/Pedigree

In [11]:
# Data description
df = pd.read_excel(files['6b_dbGaP_PedigreeDD.xlsx'], delimiter='\t', dtype={'SUBJID': str})
df.head()

,VARNAME,VARDESC,DOCFILE,TYPE,VARIABLE_SOURCE,SOURCE_VARIABLE_ID,COLLINTERVAL,VALUES,Unnamed: 8,Unnamed: 9
0,FAMILY_ID,Family_ID,NaN,encoded value,NaN,NaN,Collected at enrollment,NaN,NaN,NaN
1,SUBJECT_ID,De-identified Subject ID,NaN,encoded value,NaN,NaN,Collected at enrollment,NaN,NaN,NaN
2,Father,ID of father,NaN,encoded value,NaN,NaN,Collected at enrollment,NaN,NaN,NaN
3,Mother,ID of mother,NaN,encoded value,NaN,NaN,Collected at enrollment,NaN,NaN,NaN
4,SEX,Gender of participant,NaN,encoded value,NaN,NaN,Collected at enrollment,M=male,F=female,U=unknown


In [12]:
df = pd.read_excel(files['6a_dbGaP_PedigreeDS_corrected.6.12.xlsx'], delimiter='\t', dtype={'SUBJID': str})
df.head()

,Family_id,SUBJECT_ID,Father,Mother,SEX
0,06-0015,06-0015,6-15F,6-15M,F
1,07-0016,07-0016,CDH1377,7-16M,M
2,09-0019,09-0019,CDH1247,CDH1246,M
3,06-0015,6-15F,NaN,NaN,M
4,06-0015,6-15M,NaN,NaN,F


In [13]:
df.describe(include=['O']).T.sort_values('unique', ascending=False)

,count,unique,top,freq
SUBJECT_ID,962,962,CDH14-7F,1
Family_id,962,321,168,3
Father,320,320,CDH179,1
Mother,320,320,CDH5-68M,1
SEX,962,3,M,518


### Phenotypes

In [14]:
# Data description
df = pd.read_excel(files['2b_dbGaP_SubjectPhenotypesDD.xlsx'], delimiter='\t', dtype={'SUBJID': str})
df

,VARNAME,VARDESC,DOCFILE,TYPE,VARIABLE_SOURCE,SOURCE_VARIABLE_ID,COLLINTERVAL,VALUES,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,SUBJECT_ID,De-identified Subject ID,NaN,string,NaN,NaN,Collected at enrollment,NaN,NaN,NaN,NaN,NaN
1,SEX,Gender of participant,NaN,encoded value,NaN,NaN,Collected at enrollment,M=Male,F=Female,U=unknown,NaN,NaN
2,ISOLATED,If the subject has a second birth defect,NaN,encoded value,NaN,NaN,Collected at enrollment,1=Isolated,2=Non-isolated,U=unknown,NA=Not applicable,NaN
3,CHD,If the subject has a congenital heart defect,NaN,encoded value,NaN,NaN,Collected at enrollment,1=yes,0=No,NA=Not applicable,NaN,NaN
4,CNS,If the subject has a central nervous system defect,NaN,encoded value,NaN,NaN,Collected at enrollment,1=yes,0=No,NA=Not applicable,NaN,NaN
5,GI,If the subject has a gastrointestinal defect,NaN,encoded value,NaN,NaN,Collected at enrollment,1=yes,0=No,NA=Not applicable,NaN,NaN
6,other_cong_malf_1,Other birth defects (1),NaN,encoded value,NaN,NaN,Collected at enrollment,text,0=No,NA=Not applicable,NaN,NaN
7,other_cong_malf_2,Other birth defects (2),NaN,encoded value,NaN,NaN,Collected at enrollment,text,0=No,NA=Not applicable,NaN,NaN
8,other_cong_malf_3,Other birth defects (3),NaN,encoded value,NaN,NaN,Collected at enrollment,text,0=No,NA=Not applicable,NaN,NaN
9,discharge_status,Discharge status,NaN,encoded value,NaN,NaN,Collected at enrollment,1=Alive,4=Deceased,0=Fetal sample,8=unknown,NA=Not applicable


In [15]:
# Data
df = pd.read_excel(files['2a_dbGaP_SubjectPhenotypesDS.xlsx'], delimiter='\t', dtype={'SUBJID': str})
df.head()

,SUBJECT_ID,SEX,ISOLATED,CHD,CNS,GI,other_cong_malf_1,other_cong_malf_2,other_cong_malf_3,discharge_status,Ethnicity,Race
0,06-0015,F,2,0.0,0.0,0.0,congenital pulmonary sequestration malformation,0,0,1.0,non-Hispanic,white
1,07-0016,M,1,0.0,0.0,0.0,0,0,0,1.0,non-Hispanic,white
2,09-0019,M,2,1.0,1.0,0.0,pulmonary hypoplasia,"Facial & Body deformities suggestive of Potter sequence: wide inter brain weight, canthal distance; flattened, widened nose; micrognathia; flattened pinnae w/ somewhat bulbous earlobes; varus deformity with metatarsus adductus, feet, severe",small kidneys; renal proximal tubular epithelial degeneration and sloughing with cortical medullary junction congestion,4.0,non-Hispanic,white
3,6-15F,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non-Hispanic,white
4,6-15M,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non-Hispanic,white


In [16]:
df.describe(include=['O']).T.sort_values('unique', ascending=False)

,count,unique,top,freq
SUBJECT_ID,962,962,CDH14-7F,1
other_cong_malf_1,321,61,0,254
other_cong_malf_2,320,25,0,296
other_cong_malf_3,319,10,0,310
Race,962,9,White,450
Ethnicity,962,5,Non-Hispanic,412
SEX,962,3,M,518
ISOLATED,323,3,1,183


### Samples

In [17]:
# Subject sample data description
filepath = os.path.join(DBGAP_DIR, '5b_dbGaP_SubjectSampleMappingDD.xlsx')
df = pd.read_excel(filepath, delimiter='\t')
df.head()

,VARNAME,VARDESC,DOCFILE,TYPE,VARIABLE_SOURCE,SOURCE_VARIABLE_ID,COLLINTERVAL,VALUES
0,SUBJECT_ID,De-identified Subject ID,NaN,encoded value,NaN,NaN,Collected at enrollment,NaN
1,SAMPLE_ID,De-identified Sample ID,NaN,encoded value,NaN,NaN,Collected at enrollment,NaN
2,SAMPLE_USE,Sample use,NaN,encoded value,NaN,NaN,Collected at enrollment,NaN


In [18]:
# Subject sample mapping
filepath = os.path.join(DBGAP_DIR, '5a_dbGaP_SubjectSampleMappingDS cumulative.xlsx')
df = pd.read_excel(filepath, delimiter='\t')
df.head()

,SUBJECT_ID,SAMPLE_ID,SAMPLE_USE
0,1,1,Seq_DNA_WholeGenome
1,2,2,Seq_DNA_WholeGenome
2,7,7,Seq_DNA_WholeGenome
3,8,8,Seq_DNA_WholeGenome
4,10,10,Seq_DNA_WholeGenome


In [19]:
df.describe(include=['O']).T.sort_values('unique', ascending=False)

,count,unique,top,freq
SUBJECT_ID,1556,1556,CDH10-6M,1
SAMPLE_ID,1556,1556,SM-DI6CU,1
SAMPLE_USE,1556,1,Seq_DNA_WholeGenome,1556


In [20]:
# Sample manifests
# Combine all sample manifest sheets
dfs = [pd.read_excel(os.path.join(MANIFESTS_DIR, filename))

       for filename in os.listdir(MANIFESTS_DIR)

       ]
df = pd.concat(dfs)
df = df[df['Sample ID'].notnull()]
print(df.shape)
df.head()

(908, 15)


,Alias,Alias.1,Alias.2,Alias.3,Collected After 01/25/2015,Concentration,Gender,RIN Number,Sample ID,Sample Type,Unnamed: 12,Unnamed: 13,Unnamed: 8,Volume,Well
1,h1258FBBc,h1258FBBc1,Father,NaN,NaN,50,Male,NaN,SM-DJT8D,DNA,NaN,NaN,NaN,70,A01
2,m1021LEMa,m1021LEMa1,Proband,NaN,NaN,50,Female,NaN,SM-DJT8E,DNA,NaN,NaN,NaN,70,A02
3,m1029QENa,m1029QENa1,Proband,NaN,NaN,50,Unknown,NaN,SM-DJT8F,DNA,NaN,NaN,NaN,30,A03
4,m1008SAWb,m1008SAWb1,Mother,NaN,NaN,50,Female,NaN,SM-DJT8G,DNA,NaN,NaN,NaN,33,A04
5,m1038MWLb,m1038MWLb1,Mother,NaN,NaN,50,Female,NaN,SM-DJT8H,DNA,NaN,NaN,NaN,49,A05


In [21]:
# Phenotype sheets
dfs = [pd.read_excel(os.path.join(MANIFESTS_DIR, filename), sheet_name=1)

       for filename in os.listdir(MANIFESTS_DIR)

       ]
df = pd.concat(dfs)
df = df[df['Sample ID'].notnull()]
# Make all values lower case
df['Primary Disease'] = df['Primary Disease'].apply(lambda x: str(x).lower())

print(df.shape)
df.head()

(908, 10)


,Age,BSP Notes PT,Ethnicity,Primary Disease,Race,Sample ID,Unnamed: 7,Unnamed: 8,Well,multiple races
1,NaN,NaN,non-Hispanic,unaffected parent,White,SM-DJT8D,h1258FBBc,h1258FBBc1,A01,NaN
2,NaN,"hypoplastic aortic arch, peristant L SVC, 2 vessel cord",non-Hispanic,congenital diaphragmatic hernia - complex,White,SM-DJT8E,m1021LEMa,m1021LEMa1,A02,NaN
3,NaN,NaN,unknown,"congenital diaphragmatic hernia - complex, poc",White,SM-DJT8F,m1029QENa,m1029QENa1,A03,NaN
4,NaN,NaN,non-Hispanic,unaffected parent,White,SM-DJT8G,m1008SAWb,m1008SAWb1,A04,NaN
5,NaN,NaN,non-Hispanic,unaffected parent,White,SM-DJT8H,m1038MWLb,m1038MWLb1,A05,NaN


## Extract

#### Participants, Family Relationships

In [22]:
subject_df = read_subject_data()
subject_df.head()

,subject_id,consent,affected_status
0,06-0015,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",affected
1,07-0016,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",affected
2,09-0019,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",affected
3,6-15F,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",unaffected
4,6-15M,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",unaffected


In [23]:
subject_attr_df = read_subject_attr_data()
subject_attr_df.head()

,subject_id,sample_id,body_site,analyte_type,is_tumor
0,06-0015,SM-DH4NH,diaphragm,DNA,N
1,07-0016,SM-DH4NJ,blood,DNA,N
2,09-0019,SM-DH4NK,skin,DNA,N
3,6-15F,SM-DH4NF,blood,DNA,N
4,6-15M,SM-DH4NG,blood,DNA,N


In [24]:
family_df = read_family_data()
family_df.head()

,family_id,subject_id,father,mother
0,06-0015,06-0015,6-15F,6-15M
1,07-0016,07-0016,CDH1377,7-16M
2,09-0019,09-0019,CDH1247,CDH1246
3,06-0015,6-15F,None,None
4,06-0015,6-15M,None,None


In [25]:
sample_manifest_df = read_sample_manifests()
sample_manifest_df.head()

,concentration,volume,sample_id,sample_type,is_proband
1,50,70,SM-DJT8D,DNA,Father
2,50,70,SM-DJT8E,DNA,Proband
3,50,30,SM-DJT8F,DNA,Proband
4,50,33,SM-DJT8G,DNA,Mother
5,50,49,SM-DJT8H,DNA,Mother


In [27]:
# Participant df
# Merge subject + subject attributes
df1 = pd.merge(subject_df, subject_attr_df, on='subject_id')
df1.head()

# Merge family
df2 = pd.merge(df1, family_df, on='subject_id')
print('{} Participants w/o samples merged'.format(df2.shape))

# Merge proband from sample manifests
participant_df = pd.merge(df2, sample_manifest_df[['sample_id', 'is_proband']], on='sample_id')
print('{} Participants w samples merged'.format(participant_df.shape))
participant_df.head()

(959, 10) Participants w/o samples merged
(864, 11) Participants w samples merged


,subject_id,consent,affected_status,sample_id,body_site,analyte_type,is_tumor,family_id,father,mother,is_proband
0,06-0015,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",affected,SM-DH4NH,diaphragm,DNA,N,06-0015,6-15F,6-15M,Proband
1,07-0016,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",affected,SM-DH4NJ,blood,DNA,N,07-0016,CDH1377,7-16M,Proband
2,09-0019,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",affected,SM-DH4NK,skin,DNA,N,09-0019,CDH1247,CDH1246,Proband
3,6-15F,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",unaffected,SM-DH4NF,blood,DNA,N,06-0015,None,None,Father
4,6-15M,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",unaffected,SM-DH4NG,blood,DNA,N,06-0015,None,None,Mother


#### Demographic

In [28]:
demographic_df = read_demographic_data()
demographic_df = pd.merge(demographic_df, participant_df, on='subject_id')
demographic_df.head()

,subject_id,sex,ethnicity,race,consent,affected_status,sample_id,body_site,analyte_type,is_tumor,family_id,father,mother,is_proband
0,06-0015,F,non-hispanic,white,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",affected,SM-DH4NH,diaphragm,DNA,N,06-0015,6-15F,6-15M,Proband
1,07-0016,M,non-hispanic,white,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",affected,SM-DH4NJ,blood,DNA,N,07-0016,CDH1377,7-16M,Proband
2,09-0019,M,non-hispanic,white,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",affected,SM-DH4NK,skin,DNA,N,09-0019,CDH1247,CDH1246,Proband
3,6-15F,M,non-hispanic,white,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",unaffected,SM-DH4NF,blood,DNA,N,06-0015,None,None,Father
4,6-15M,F,non-hispanic,white,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",unaffected,SM-DH4NG,blood,DNA,N,06-0015,None,None,Mother


#### Samples

In [32]:
# Subject sample mappings
subject_sample_df = read_subject_sample_data()
subject_sample_df.head()

# Merge with subject data
df3 = pd.merge(participant_df, subject_sample_df[['subject_id', 'sample_use']], on='subject_id')
print(df.shape)
df.head()

# Merge with sample manifests
sample_df = pd.merge(df3, sample_manifest_df, on='sample_id')
print(sample_df.shape)
sample_df.head()

(908, 10)
(864, 16)


,subject_id,consent,affected_status,sample_id,body_site,analyte_type,is_tumor,family_id,father,mother,is_proband_x,sample_use,concentration,volume,sample_type,is_proband_y
0,06-0015,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",affected,SM-DH4NH,diaphragm,DNA,N,06-0015,6-15F,6-15M,Proband,Seq_DNA_WholeGenome,50ng/uL,30uL,DNA,Proband
1,07-0016,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",affected,SM-DH4NJ,blood,DNA,N,07-0016,CDH1377,7-16M,Proband,Seq_DNA_WholeGenome,50ng/uL,30uL,DNA,Proband
2,09-0019,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",affected,SM-DH4NK,skin,DNA,N,09-0019,CDH1247,CDH1246,Proband,Seq_DNA_WholeGenome,50ng/uL,30uL,DNA,Proband
3,6-15F,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",unaffected,SM-DH4NF,blood,DNA,N,06-0015,None,None,Father,Seq_DNA_WholeGenome,50ng/uL,30uL,DNA,Father
4,6-15M,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",unaffected,SM-DH4NG,blood,DNA,N,06-0015,None,None,Mother,Seq_DNA_WholeGenome,50ng/uL,30uL,DNA,Mother


#### Phenotype

In [39]:
# Read phenotype
phenotype_df = read_phenotype_data()
phenotype_df.head()

# Merge with participant df
phenotype_df = pd.merge(phenotype_df, participant_df, on='subject_id')
phenotype_df.head()

,subject_id,phenotype,observed,hpo_id,consent,affected_status,sample_id,body_site,analyte_type,is_tumor,family_id,father,mother,is_proband
0,06-0015,congenital heart defect,negative,None,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",affected,SM-DH4NH,diaphragm,DNA,N,06-0015,6-15F,6-15M,Proband
1,06-0015,central nervous system defect,negative,None,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",affected,SM-DH4NH,diaphragm,DNA,N,06-0015,6-15F,6-15M,Proband
2,06-0015,gastrointestinal defect,negative,None,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",affected,SM-DH4NH,diaphragm,DNA,N,06-0015,6-15F,6-15M,Proband
3,06-0015,congenital pulmonary sequestration malformation,positive,None,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",affected,SM-DH4NH,diaphragm,DNA,N,06-0015,6-15F,6-15M,Proband
4,06-0015,congenital birth defect,negative,None,"Disease-Specific (Congenital Diaphragmatic Hernia, COL, GSO, RD) (DS-CDH-COL-GSO-RD)",affected,SM-DH4NH,diaphragm,DNA,N,06-0015,6-15F,6-15M,Proband


In [36]:
outcome_df = read_outcome_data()
outcome_df.head()

,subject_id,discharge_status
0,06-0015,deceased
1,07-0016,deceased
2,09-0019,fetal sample
3,6-15F,not applicable
4,6-15M,not applicable
